In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpcful packages to load
import os 
import sys
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import random
import csv
import cv2
import math
import PIL
from collections import namedtuple, OrderedDict
import io
from PIL import Image
from collections import namedtuple, OrderedDict

%matplotlib inline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
DATA_DIR = '../input/airbus-ship-detection/' 
ROOT_DIR = '/kaggle/working'
os.chdir(ROOT_DIR)

# **Training data**

In [ ]:
train_v2_list = os.listdir(DATA_DIR + 'train_v2')

In [ ]:
train_df = pd.read_csv(DATA_DIR + "train_ship_segmentations_v2.csv")
train_df

In [ ]:
train_df['ShipCount'] = train_df.groupby('ImageId')['ImageId'].transform('count')
train_df.loc[train_df['EncodedPixels'].isnull().values,'ShipCount'] = 0

In [ ]:
train_df

In [ ]:
count_df = train_df.groupby('ShipCount').count()
count_df

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(count_df.index.values.tolist(), list(count_df['ImageId']))
plt.show()

# Image Enhancement

In [ ]:
sampleList = ['001aee007.jpg','001234638.jpg','001f04ca3.jpg','000d26c17.jpg']
sampleImgList = []
for x in sampleList:
    sampleImgList.append(mpimg.imread(DATA_DIR + 'train_v2/' + x))

In [ ]:
fig = plt.figure(1,figsize=(20,10))
for i in range(len(sampleImgList)):
    image_tmp = sampleImgList[i]
    ax = fig.add_subplot(1,4,i+1)
    ax.imshow(image_tmp)

In [ ]:
from skimage.feature import canny
from skimage.filters import scharr, unsharp_mask
from skimage import exposure
from skimage.color.adapt_rgb import adapt_rgb, each_channel, hsv_value

In [ ]:
fig = plt.figure(1,figsize=(20,20))
for i in range(len(sampleImgList)):
    image_tmp = sampleImgList[i]
    ax = fig.add_subplot(2,4,i+1)
    ax.imshow(image_tmp)
    image_tmp = unsharp_mask(sampleImgList[i], radius=4, amount=2)
    sampleImgList[i] = image_tmp
    ax = fig.add_subplot(1,4,i+1)
    ax.imshow(image_tmp)

# Decoding the pixels

In [ ]:
def rle_to_pixels(rle_code):
    '''
    Transforms a RLE code string into a list of pixels of a (768, 768) canvas
    '''
    rle_code = [int(i) for i in rle_code.split()]
#     pixels = [(pixel_position % 768, pixel_position // 768) 
#                  for start, length in list(zip(rle_code[0:-1:2], rle_code[1::2])) 
#                  for pixel_position in range(start, start + length)]
    
    pixels = []
    for start, length in list(zip(rle_code[0:-1:2], rle_code[1::2])):
        for pixel_position in range(start, start + length):
            pixels.append((pixel_position % 768, pixel_position // 768))
            #if pixel_position < 0 or pixel_position > 768:
                #print(pixel_position)
    
    return pixels

def get_all_masks(image_id):
    ret = []
    s = train_df[train_df['ImageId'] == image_id]['EncodedPixels']
    if not s.isnull().values.any():
        for x in s:
            ret.append(rle_to_pixels(x))
    return ret

def show_masks(image_id):
    canvas = np.zeros((768, 768))
    masks = get_all_masks(image_id)
    for x in masks:
       canvas[tuple(zip(*x))] = 1
    return canvas

def find_bounding_box(pixels):
    xmin = 767
    xmax = 1
    ymin = 767
    ymax = 1
    for p in pixels:
        px = p[0]
        py = p[1]
        if px !=0 and py != 0 and px !=768 and py != 768:
            if px < xmin:
                xmin = px
            if px > xmax:
                xmax = px
            if py < ymin:
                ymin = py
            if py > ymax:
                ymax = py
    return xmin, ymin, xmax, ymax

def get_all_boxes(image_id):
    ret = []
    masks = get_all_masks(image_id)
    for x in masks:
        ret.append(find_bounding_box(x))
    return ret

def show_bounding_box(image_id):
    canvas = np.array(PIL.Image.open(DATA_DIR + 'train_v2/' + image_id))
    boxes = get_all_boxes(image_id)
    for x in boxes:
        xmin, ymin, xmax, ymax = x[0],x[1], x[2], x[3]
        canvas[xmin][ymin : ymax] = [0,255,0]
        canvas[xmax][ymin : ymax] = [0,255,0]
        canvas[:,ymin][xmin : xmax] = [0,255,0]
        canvas[:,ymax][xmin : xmax] = [0,255,0]
    return canvas

In [ ]:
fig = plt.figure(1,figsize=(20,20))
for i in range(len(sampleList)): 
    image_tmp = show_masks(sampleList[i])
    ax = fig.add_subplot(1,4,i+1)
    ax.imshow(image_tmp)
    image_tmp = show_bounding_box(sampleList[i])
    ax = fig.add_subplot(2,4,i+1)
    ax.imshow(image_tmp)

# Tensorflow Object Detection API setup

In [ ]:
!pip uninstall -y tensorflow
!pip uninstall -y tensorflow-gpu
!pip uninstall -y tensorflow-estimator

In [ ]:
#Change TF version
!pip list | grep tensorflow
!pip install tensorflow-gpu==1.15 #1.15
# !pip uninstall -y tensorflow==2.2
# !pip list | grep tensorflow
!pip install tensorflow-estimator==1.15
!pip list | grep tensorflow

In [ ]:
import os
os.getcwd()

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

In [ ]:
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

In [ ]:
!pip install tensorflow-object-detection-api==0.1.0 --no-dependencies

In [ ]:
os.chdir(ROOT_DIR)
!git clone https://github.com/tensorflow/models.git
#!git clone https://github.com/tensorflow/models/archive/v2.2.0.zip
# !wget  https://github.com/tensorflow/models/archive/v2.2.0.zip

In [ ]:
os.chdir(ROOT_DIR)

In [ ]:
ls

In [ ]:
!apt-get -y install protobuf-compiler
!pip install Cython
!pip install pillow
!pip install lxml
!pip install jupyter
!pip install matplotlib
!pip install tf_slim

In [ ]:
os.chdir(ROOT_DIR+"/models/research/")
!protoc object_detection/protos/*.proto --python_out=.
!export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim
os.environ['PYTHONPATH'] += ':/kaggle/working/models/research/:/kaggle/working/models/research/slim/:/kaggle/working/models'

In [ ]:
os.getcwd()

In [ ]:
#Test if set up is successful
!python ./models/research/object_detection/builders/model_builder_test.py
# !python object_detection/builders/model_builder_tf2_test.py

In [ ]:
os.chdir(ROOT_DIR)

# LabelMap

In [ ]:
with open(ROOT_DIR+'/labelmap.pbtxt', 'w+') as the_file:
    the_file.write('item\n')
    the_file.write('{\n')
    the_file.write('id :{}'.format(int(1)))
    the_file.write('\n')
    the_file.write("name :'{0}'".format('ship'))
    the_file.write('\n')
    the_file.write('}\n')
    the_file.close()

# Generate annotations

## Generate xml files

In [ ]:
import xml.etree.cElementTree as ET
def generate_xml(imageId):
    annotation = ET.Element("annotation")
    ET.SubElement(annotation, "folder").text = "train_v2"
    ET.SubElement(annotation, "filename").text = imageId
    source = ET.SubElement(annotation, "source")
    ET.SubElement(source, "database").text = "Unknown"
    size = ET.SubElement(annotation, "size")
    ET.SubElement(size, "width").text = "768"
    ET.SubElement(size, "height").text = "768"
    ET.SubElement(size, "depth").text = "3"
    ET.SubElement(annotation, "segmented").text = "0"
    
    boxes = get_all_boxes(imageId)
    for b in boxes:
        object1 = ET.SubElement(annotation, "object")
        ET.SubElement(object1, "name").text = "ship"
        ET.SubElement(object1, "name").text = "ship"
        ET.SubElement(object1, "pose").text = "Unspecified"
        ET.SubElement(object1, "truncated").text = "0"
        ET.SubElement(object1, "difficult").text = "0"
        bndbox = ET.SubElement(object1, "bndbox")
        xmin, ymin, xmax, ymax = b
        ET.SubElement(bndbox, "xmin").text = str(xmin)
        ET.SubElement(bndbox, "ymin").text = str(ymin)
        ET.SubElement(bndbox, "xmax").text = str(xmax)
        ET.SubElement(bndbox, "ymax").text = str(ymax)

    tree = ET.ElementTree(annotation)
    tree.write("test.xml")

## Generate dataframe for TFRecords

In [ ]:
data = []
###############################################DEBUG
for x in train_df['ImageId'][:100]:
    boxes = get_all_boxes(x)
    for b in boxes:
        xmin, ymin, xmax, ymax = b
        data.append((x, 768, 768, 'ship', xmin, ymin, xmax, ymax))
columns_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
TFRcords_df = pd.DataFrame(data=data, columns=columns_name)
TFRcords_df

# Training & validation split

In [ ]:
#A naive way to split for code testing

train_set = TFRcords_df[0:20]
valid_set = TFRcords_df[21:25]

In [ ]:
# pip install tf-nightly

In [ ]:
sys.path.append("..")
from models.research.object_detection.utils import dataset_util
from models.research.object_detection.utils import label_map_util

In [ ]:
# Function to group data and return the same
# Group by imagefile name
def make_groups(df, field=None):
    if field==None:
        field = 'filename'

    data = namedtuple('object', ['filename', 'info'])
    grouped = df.groupby(field)

    grouped_data = []
    for filename, x in zip(grouped.groups.keys(), grouped.groups):
        grouped_data.append(data(filename, grouped.get_group(x)))

    return grouped_data


In [ ]:
# Creating a tf record sample
def create_tf_example(group, img_path, label_map_dict):
    # Read the imagefile. This will be used in features later 
    with tf.io.gfile.GFile(os.path.join(img_path, '{}'.format(group.filename)), 'rb') as f:
        img_file = f.read()

        # Encode to bytes and read using PIL. Could be done directly too
        encoded_img = io.BytesIO(img_file)
        # Read the image using PIL
        img = Image.open(encoded_img)
        width, height = img.size

      # Encode the name of the img file
        filename = group.filename.encode('utf8')

      # Define the format of the image file
        img_format = b'jpg'   # The name will be in bytes


      # Define the variables that you need as features
        xmins = []
        xmaxs = []
        ymins = []
        ymaxs = []
        classes_text = []
        classes = []

      # Iterate over the namedtuple object
        for index, row in group.info.iterrows():
            xmins.append(row['xmin'] / width)   # store normalized values for bbox
            xmaxs.append(row['xmax'] / width)
            ymins.append(row['ymin'] / height)
            ymaxs.append(row['ymax'] / height)
            classes_text.append(row['class'].encode('utf8'))
            classes.append(label_map_dict[row['class']])

        tf_example = tf.train.Example(features=tf.train.Features(feature={
          'image/height': dataset_util.int64_feature(height),
          'image/width': dataset_util.int64_feature(width),
          'image/filename': dataset_util.bytes_feature(filename),
          'image/source_id': dataset_util.bytes_feature(filename),
          'image/encoded': dataset_util.bytes_feature(img_file),
          'image/format': dataset_util.bytes_feature(img_format),
          'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
          'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
          'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
          'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
          'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
          'image/object/class/label': dataset_util.int64_list_feature(classes),}))

        return tf_example

## Create TFRecord Files

In [ ]:
# Path where all the images are present
img_path = DATA_DIR + 'train_v2'
# Label map
label_map_dict = label_map_util.get_label_map_dict(ROOT_DIR + '/labelmap.pbtxt')

writer = tf.compat.v1.python_io.TFRecordWriter('./train.record')

# create groups in the df. One image may contain several instances of an object hence the grouping thing
img_groups = make_groups(train_set, field='filename')
# Iterate over the samples in each group create a TFRecord
for group in img_groups:
    tf_example = create_tf_example(group, img_path, label_map_dict)
    writer.write(tf_example.SerializeToString())
# close the writer
writer.close()
print("TFRecords for training data  created successfully")


writer = tf.compat.v1.python_io.TFRecordWriter('./valid.record')
# create groups 
img_groups = make_groups(valid_set, field='filename')
# Iterate over the samples in each group create a TFRecord
for group in img_groups:
    tf_example = create_tf_example(group, img_path, label_map_dict)
    writer.write(tf_example.SerializeToString())
# close the writer
writer.close()
print("TFRecords for validation data created successfully")

# Import the model

In [ ]:
#!cp /kaggle/working/models/research/object_detection/samples/configs/ssd_inception_v2_coco.config /kaggle/working

#!cp /kaggle/working/models/research/object_detection/samples/configs/faster_rcnn_inception_v2_coco.config /kaggle/working
!cp /kaggle/working/models/research/object_detection/samples/configs/faster_rcnn_inception_resnet_v2_atrous_coco.config /kaggle/working

In [ ]:
#!wget download.tensorflow.org/models/object_detection/ssd_inception_v2_coco_2017_11_17.tar.gz
#!wget download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8.tar.gz
#!wget download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8.tar.gz
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8.tar.gz

In [ ]:
#!tar -xzf ssd_inception_v2_coco_2017_11_17.tar.gz
#!tar -xzf faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8.tar.gz
!tar -xzf faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8.tar.gz


In [ ]:
#!mv ssd_inception_v2_coco_2017_11_17 mymodel
#!mv faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8 mymodel
!mv faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8 mymodel

In [ ]:
!pip install Cython
!git clone https://github.com/pdollar/coco.git
os.chdir('coco/PythonAPI')
!make
!make install
!python setup.py install
os.chdir(ROOT_DIR)


# Configure the model config file

In [ ]:
os.chdir(ROOT_DIR)
os.getcwd()

In [ ]:
# Configures the .config automatically
#fin = open("ssd_inception_v2_coco.config", "rt")
#fin = open("faster_rcnn_inception_v2_coco.config", "rt")
fin = open("faster_rcnn_inception_resnet_v2_atrous_coco.config", "rt")

fout = open("configfile.config", "wt")


for line in fin:
    if 'num_classes:' in line:
        fout.write('\t\tnum_classes: 1\n')
    else:
        line = line.replace('PATH_TO_BE_CONFIGURED/model.ckpt', '/kaggle/working/mymodel/model.ckpt')
        line = line.replace('PATH_TO_BE_CONFIGURED/mscoco_train.record-?????-of-00100', '/kaggle/working/train.record')
        line = line.replace('PATH_TO_BE_CONFIGURED/mscoco_label_map.pbtxt', '/kaggle/working/labelmap.pbtxt')
        line = line.replace('PATH_TO_BE_CONFIGURED/mscoco_val.record-?????-of-00010','/kaggle/working/valid.record')
        line = line.replace('num_steps: 200000','num_steps: 5000')
        fout.write(line)

fin.close()
fout.close()

# Train the model

In [ ]:
!mkdir checkpoints
!cp /kaggle/working/models/research/object_detection/legacy/train.py /kaggle/working
!cp /kaggle/working/models/research/object_detection/legacy/eval.py /kaggle/working
!cp /kaggle/working/models/research/object_detection/export_inference_graph.py /kaggle/working
!cp /kaggle/working/models/research/object_detection/model_main.py /kaggle/working

In [ ]:
tf.__version__

In [ ]:
os.chdir(ROOT_DIR)

In [ ]:
os.mkdir("Output_point")

In [ ]:
#run the training
# import tensorflow.compat.v2 as tf
# !python train.py --logtostderr --train_dir=/kaggle/working/checkpoints/ --pipeline_config_path=/kaggle/working/configfile.config
!python model_main.py  --logtostderr --model_dir=/kaggle/working/Output_point/ --pipeline_config_path=/kaggle/working/configfile.config

In [ ]:
!pip list | grep tensorflow

In [ ]:
!mkdir /kaggle/working/trained

In [ ]:
# os.listdir("/kaggle/working/checkpoints/")
os.listdir("/kaggle/working/Output_point/")

In [ ]:
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path /kaggle/working/configfile.config --trained_checkpoint_prefix ./Output_point/model.ckpt-50 --output_directory /kaggle/working/trained

In [ ]:
!ls -al /kaggle/working/trained

In [ ]:
# !tar -cvzf /kaggle/working/trained_model.tar /kaggle/working/trained
# !gzip -y /kaggle/working/trained_model.tar

# Test Data


In [ ]:
test_v2_list = os.listdir(DATA_DIR + 'test_v2')

In [ ]:
test_df = pd.read_csv(DATA_DIR + "sample_submission_v2.csv")

In [ ]:
test_df

# Prediction

In [ ]:
print(os.getcwd())
!ls

In [ ]:
# # List of the strings that is used to add correct label for each box.
# PATH_TO_LABELS = '/kaggle/working/labelmap.pbtxt'
# category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [ ]:
def clmpc(label_map, max_num_classes,use_display_name=True):
    categories = []
    list_of_ids_already_added = []
    if not label_map:
        label_id_offset = 1
        for class_id in range(max_num_classes):
            categories.append({
                'id': class_id + label_id_offset,
                'name': 'category_{}'.format(class_id + label_id_offset)})
        return categories
    for item in label_map.item:
        if not 0 < item.id <= max_num_classes:
#             logging.info('Ignore item %d since it falls outside of requested ''label range.', item.id)
            continue
        if use_display_name and item.HasField('display_name'):
            name = item.display_name
        else:
            name = item.name
        if item.id not in list_of_ids_already_added:
            list_of_ids_already_added.append(item.id)
            categories.append({'id': item.id, 'name': name})
    return categories

In [ ]:
# What model to download.
MODEL_NAME = 'trained'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = '/kaggle/working/labelmap.pbtxt'

NUM_CLASSES = 1

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [ ]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = clmpc(label_map,max_num_classes=NUM_CLASSES)

category_index = label_map_util.create_category_index(categories)

In [ ]:
print(category_index)

In [ ]:
def clmpc(label_map, max_num_classes,use_display_name=True):
    categories = []
    list_of_ids_already_added = []
    if not label_map:
        label_id_offset = 1
        for class_id in range(max_num_classes):
            categories.append({
                'id': class_id + label_id_offset,
                'name': 'category_{}'.format(class_id + label_id_offset)})
        return categories
    for item in label_map.item:
        if not 0 < item.id <= max_num_classes:
#             logging.info('Ignore item %d since it falls outside of requested ''label range.', item.id)
            continue
        if use_display_name and item.HasField('display_name'):
            name = item.display_name
        else:
            name = item.name
        if item.id not in list_of_ids_already_added:
            list_of_ids_already_added.append(item.id)
            categories.append({'id': item.id, 'name': name})
    return categories

In [ ]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
PATH_TO_TEST_IMAGES_DIR = '../input/airbus-ship-detection/test_v2/'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR,i ) for i in test_df['ImageId'][10:20]]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)
print(TEST_IMAGE_PATHS)

In [ ]:
def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in ['num_detections', 'detection_boxes', 'detection_scores','detection_classes', 'detection_masks']:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                    # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

          # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict['detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
        if 'detection_masks' in output_dict:
            output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

In [ ]:
from object_detection.utils import visualization_utils as vis_util
for image_path in TEST_IMAGE_PATHS:
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)